In [1]:
import cv2
import pytesseract

# Specify the path to tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the image of the vehicle
image = cv2.imread('car_image.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply GaussianBlur to reduce noise and improve contour detection
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Perform edge detection
edged = cv2.Canny(blurred, 30, 200)

# Find contours in the image
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by area and take the largest ones
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

# Loop through the contours to find a rectangle (license plate shape)
plate_contour = None
for contour in contours:
    # Approximate the contour
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
    
    # Check if the contour has 4 points (a rectangle)
    if len(approx) == 4:
        plate_contour = approx
        break

# If license plate contour is found, draw it on the image
if plate_contour is not None:
    cv2.drawContours(image, [plate_contour], -1, (0, 255, 0), 3)

# Extract the region of interest (ROI) where the plate is located
x, y, w, h = cv2.boundingRect(plate_contour)
plate_image = gray[y:y+h, x:x+w]

# Use Tesseract to perform OCR on the license plate
text = pytesseract.image_to_string(plate_image, config='--psm 8')

# Display the recognized text
print("Detected license plate Number is:", text)

# Display the output image with detected license plate contour
cv2.imshow('License Plate Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
